In [1]:
# imports
from typing import List
import random


In [5]:
# player and Card class
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
    
    def __str__(self):
        return f"Card Value: {self.value} --- Card Suit: {self.suit}"
    
class Player:
    def __init__(self, player_num: int, money: int, move: str, card: Card, is_human: bool):
        self.player_num = player_num
        self.money = money
        self.move = move
        self.card = card
        self.is_human = is_human
        self.can_make_move = True
                
    
    def __str__(self):
        return f"Player Num: {self.player_num} -- Player Money: {self.money} -- Player Move: {self.move} -- Current Card: {self.card} "

    


In [3]:
# setup for players inputs

valid_bid_list = ["b", "Bid", "bid", "B"]
valid_fold_list = ["f", "Fold", "fold", "F"]
valid_pass_list = [ "p", "Pass", "pass" ,"P" ]


def get_valid_bid_range(player: Player, bid_list: [int]) -> (int, int):
    '''
    returns the valid range that a player can bid
    '''
    
    bid_range = (0,0)
    
    # get max num from bids list
    max_bid = max(bid_list)
    
    # get how much money a player has
    player_money = player.money
    
    # if player has less money than largest bid amount in turn
    if player_money < max_bid:
        # bid range for player is limited
        bid_range = (player_money, player_money)
    else:
        # bid range is from max_bid to what player has
        bid_range = (max_bid, player_money)
        
    return bid_range
        

def player_chosen_move(player: Player, bid_amounts: [int]) -> (str, int):
    ''' 
    decised player move based on whether they are human or computer
    '''
        
    move = ("", 0)
    
    if player.is_human:
        move = get_human_input(player= player, bid_list= bid_amounts)
        
    else:
        move = get_computer_input(player= player, bid_list= bid_amounts)
        
    return move


def get_human_input(player: Player, bid_list: [int]) -> (str, int):
    '''
    asks human for valid move input
    '''
    player_move_input = input("Bid, Pass or Fold?")

    
    player_move_amount = ("", 0)
    
    # setting up how muhc a person can bid
    bid_range = get_valid_bid_range(player=player, bid_list=bid_list)
    
    
    # ask player for move 
    while player_move_input not in valid_bid_list + valid_fold_list + valid_pass_list:
        print("not valid move")
        player_move_input = input("Bid, Pass or Fold?")
        

      # if player wants to bid
    if player_move_input in valid_bid_list:
            
        # ask player for bid amount
        print("you can only bid from", bid_range[0], "to", bid_range[1])
        player_bid_input = int(input("how much do you want to bid?"))
            
        # while player inputs bid outside of range ask again
        while player_bid_input < bid_range[0] or player_bid_input > bid_range[1]:
            print("invalid bid amount")
            print("you can only bid from", bid_range[0], "to", bid_range[1])
            player_bid_input = int(input("how much do you want to bid?"))
        
        player_move_amount = (player_move_input, player_bid_input)
            
    elif player_move_input in valid_pass_list:
        print("player passed")
        player_move_amount = (player_move_input, 0)
        
    else:
        print("player folded") 
        player_move_amount = (player_move_input, 0)
             
            
    return player_move_amount


def get_computer_input(player: Player, bid_list: [int]) -> (str, int):
    return ("pass", 0)

In [ ]:
# Test Get Valid Betting Range

# creation of players
test_player1 = Player(player_num = 0, money = 5, move = "", card = None, is_human = True)

test_player2 = Player(player_num = 0, money = 5, move = "", card = None, is_human = True)

# list of players
test_players = [test_player1, test_player2]

# creation of game that takes in players
test_game = Bids(players)

# prints game state
test_game.print_state()

Player Num: 0 -- Player Money: 5 -- Player Move:  -- Current Card: Card Value: 5 --- Card Suit: j 
Player Num: 0 -- Player Money: 5 -- Player Move:  -- Current Card: Card Value: 5 --- Card Suit: j 
money in pot: 0
size of deck: 52


In [ ]:
# Bids class
class Bids:
    
    def __init__(self, players: [Player]):
        
        assert len(players) >= 2, "you need more than two players"
        self.players = players
        self.pot = 0
        self.deck = self.generate_cards()
        self.turn_bid_amounts = self.initialize_bid_list(self.players)
        
        # generates the cards to be used in a game
    def generate_cards(self) -> List[Card]:
        l_of_values = [2,3,4,5,6,7,8,9,10,11,12,13,14]
        l_of_suites = ["clubs","diamonds","hearts","spades"]
        
        deck = []
        
        for val in l_of_values:
            for suit in l_of_suites:
                curr_card = Card(val, suit)
                deck.append(curr_card)
        
        return deck
    
    # initializes bid list to all 0
    def initialize_bid_list(self,players: [Player]) -> [int]:
        bid_list = []
        for player in players:
            bid_list.append(0)
        
        return bid_list

    def give_cards_to_players(self) -> None:
        '''
        at the beggining of each round every player that has money will be given a Card. This will cost them 1 unit. 
        '''
        for player in self.players:
            # if players have money give them a card
            if player.money > 0 and player.can_make_move:
                # they pay 1
                player.money -= 1
                
                # player receives card
                player.card = random.choice(self.deck)
                
                # card get removed from deck
                self.deck.remove(player.card)
                
                self.pot +=1 # adds money to pot
                
    def play_turn(self) -> None:
        '''
        This function will make every player make a move for each round.
        
        If a later player increases the bet, we will loop to the beginning players until they make a legal move.
        
        .Players can either call, raise, or fold
        '''
        
        # while 
        for player in self.players:
            
            # if player did not fold
            if player.can_make_move:
                
                # function will take the player information and the bidding list for the round
                player_move = player_chosen_move(player, self.turn_bid_amounts)
                
                # updates the game
                # updates player move and amount of money
                # add money to pot
                player.move = player_move[0]
                player.money -= player_move[1]
                self.pot += player_move[1]
                
                print(player)
                
            if player.move in valid_fold_list or player.money < 1:
                player.can_make_move = False
            
    
    def play_game(self):
        players_can_play_list = [player.can_make_move for player in self.players]
        players_pass_list = [player.move for player in self.players]
        number_of_turns = 0
        
        is_round_over = False
        while( not is_round_over):
            
            self.play_turn()
            players_can_play_list = [player.can_make_move for player in self.players]
            players_pass_list = [player.move for player in self.players]
            print(players_can_play_list)
            
            # round is over if all player pass or fold
            if (players_can_play_list.count(False) + players_pass_list.count("p") + players_pass_list.count("P") + players_pass_list.count("pass") == len(self.players)):
                is_round_over = True
            # round is over is there's less than 1 player that can play
            if (players_can_play_list.count(True) < 2):
                is_round_over = True
            # there is a maximum of 3 turns per round
            if number_of_turns == 3:
                is_round_over = True
        
        if(is_round_over):
            # only players who can make a move can be winners
            
            self.calculate_winner(players_can_play_list)
            
    
    # def calculate_winner(self):
        # 
                  
    def print_state(self) -> None:
        '''
        prints every players state (money, action, card)
        prints money in pot
        prints size of deck
        '''
        for player in self.players:
            print(player)
        print("money in pot:", self.pot)
        print("size of deck:", len(self.deck))


In [ ]:
# function to get player who can make a move
def get_players_who_can_make_move(players):
    players_who_can_play = []
    for player in players:
        if player.can_make_move:
            players_who_can_play.append(player)
    return players_who_can_play

In [ ]:
# Example usage
card1 = Card(value=10, suit='Hearts')
card2 = Card(value=5, suit='Spades')

player1 = Player(player_num=1, money=100, move='bet', card=card1, is_human=True)
player2 = Player(player_num=2, money=50, move='fold', card=card2, is_human=False)
player2.can_make_move = False  # This player cannot make a move

players = [player1, player2]
players_who_can_play = get_players_who_can_make_move(players)


assert len(players_who_can_play) == 1, "only player 1 can make a move"
for player in players_who_can_play:
    print(player)

Player Num: 1 -- Player Money: 100 -- Player Move: bet -- Current Card: Card Value: 10 --- Card Suit: Hearts 


In [ ]:
# game
person = Player(player_num=0, money=15, move= "", card=None, is_human=True)
computer1 = Player(player_num=1, money= 15, move="", card= None, is_human=False)
computer2 = Player(player_num=2, money= 15, move="", card= None, is_human=False)


players_in_game = [person, computer1]


game = Bids(players_in_game)
game.print_state()
print()
game.give_cards_to_players()
game.print_state()


game.play_game()

Player Num: 0 -- Player Money: 15 -- Player Move:  -- Current Card: None 
Player Num: 1 -- Player Money: 15 -- Player Move:  -- Current Card: None 
money in pot: 0
size of deck: 52

Player Num: 0 -- Player Money: 14 -- Player Move:  -- Current Card: Card Value: 2 --- Card Suit: spades 
Player Num: 1 -- Player Money: 14 -- Player Move:  -- Current Card: Card Value: 14 --- Card Suit: clubs 
money in pot: 2
size of deck: 50
player folded
Player Num: 0 -- Player Money: 14 -- Player Move: f -- Current Card: Card Value: 2 --- Card Suit: spades 
Player Num: 1 -- Player Money: 14 -- Player Move: pass -- Current Card: Card Value: 14 --- Card Suit: clubs 
[False, True]
